In [1]:
import pandas as pd
from ollama import chat
from ollama import ChatResponse
from tqdm import tqdm

In [2]:
df = pd.read_csv('Drug-ATC.csv')
SMILES = df['Neutralized SMILES']
ATC4 = df['ATC Codes']

In [ ]:
s_smiles = SMILES[1].replace("\\", "\\\\") #L02AE
print(s_smiles)
response: ChatResponse = chat(model='atc4-predictor4', messages=[
    {"role": "user", "content": s_smiles}
])
print(response['message']['content'])

In [ ]:
import asyncio
from ollama import AsyncClient

async def chatA():
    message = {'role': 'user', 'content': SMILES[24].replace("\\", "\\\\")}
    response = await AsyncClient().chat(model='atc4-predictor4', messages=[message])
    print(response['message']['content'])

await chatA()

In [ ]:
if "predicted_atc4" not in df.columns:
    df["predicted_atc4"] = None
import asyncio
from ollama import AsyncClient

async def chat(message):
    response = await AsyncClient().chat(model='atc4-predictor4', messages=message)
    return response['message']['content']

for i in range(len(df)):
    smiles = df.at[i, "Neutralized SMILES"]
    name = df.at[i, "Molecule Names"]
    s_smiles = smiles.replace("\\", "\\\\")
    prompt = f"{name} - {s_smiles}"
    message=[
        {"role": "user", "content": prompt}
    ]
    try:
      pred = await chat(message)
    except ollama.ResponseError as e:
      print('Error:', e.error)
      if e.status_code == 404:
        ollama.pull(model)
    print(pred)
    df.at[i, "predicted_atc4"] = pred

    if i % 10 == 0:
        df.to_csv("predicted_atc4_partial.csv", index=False)
        print("Guardado parcial")

df.to_csv("predicted_atc4-NAME+SMILES.csv", index=False)

[M01AA03]
Guardado parcial
[D07AA04]
[D07AA06]
[A03AA04]
[A71AK08]

[J01BA03]
[C01AA08]
[A97CA03]
[A10AK04]
[A11AA], [A11BD]
[A01AA02]
Guardado parcial
[A11AA], [A11BD]
[A11AA], [A11BD]
[A11AA], [A11BD]
[A06AA02]
[A11AA02] , [A11AA03]
[A11AA07] , [A11AA08]

[A1A1], [A1B1], [A1C1]
[A11AB01], [A11BD01]
[A11]
[A11AA06] , [A11AA08]
Guardado parcial
[A11AA04], [A11AA05]
[A11AA11], [A11BA08]
A07BA01
A07BA01
A41AA01
[A11AA], [A11AG]
[A11AA07]
[A1A], [A1B]
[C10]A10AA
[A01ED01]
Guardado parcial
[C05AA09] , [C05BA02]
[C05AA09] , [C05BA03]
[A02AA02]
[A11]
[A86AX03] , [A86BX02]
[A02AA02], [A07EA01]
[A01AA01] , [A01BD01]
[A06AA06]
[A01AC04]
[C07AA04]
Guardado parcial
[JOK01], [JOK01A]
[A03EA06]
[A07AA04]
[A95XA06]

[D07AA04]

[C07AA09], [C07AA18]

[J02AC01] , [J02AC02]
[A10DB08]
J08AH02
J01DA03
Guardado parcial
[A11]
[A03AA02]
[A01AA02]
[A02BA03] , [A02BA05]
[C03AA06]
[D06AX03] , [D06BB01]
[D05AB05] , [D05AB10]
[J01CA03]
[A01AA07]
A10AX03
Guardado parcial
[D06AA08] , [D06BA03]
[A02AA02]
[A02BC01] ,

In [ ]:
response: ChatResponse = chat(model='atc4-predictor4', messages=[
    {'role': 'user', 'content': 'Cc1ncc(COP(=O)(O)O)c(C=O)c1O'}, # Pyridoxal phosphate - A11HA
    {'role': 'user', 'content': 'O=C(O)CCCC[C@@H]1CCSS1'} # Lipoic acid - A16AX
])

print(response['message']['content'])

In [ ]:
response: ChatResponse = chat(model='atc4-predictor3', messages=[
    {'role': 'user', 'content': 'NCCCC[C@H](N)C(=O)O'},
])  # Lysine - B05XB

print(response['message']['content'])

In [ ]:
response: ChatResponse = chat(model='atc4-predictor3', messages=[
    {'role': 'user', 'content': 'C/C1=C2/N=C(/C=C3\\N=C(/C(C)=C4/[C@@H](CCC(N)=O)[C@](C)(CC(N)=O)[C@](C)([C@@H]5N=C1[C@](C)(CCC(=O)NC[C@@H](C)OP(=O)([O-])O[C@H]1[C@@H](O)[C@@H](n6cnc7cc(C)c(C)cc76)O[C@@H]1CO)[C@H]5CC(N)=O)N4[Co+]C#N)[C@@](C)(CC(N)=O)[C@@H]3CCC(N)=O)C(C)(C)[C@@H]2CCC(N)=O'},
]) # Vitamin B12 - B03AE; B03BA

print(response['message']['content'])